In [10]:
import ee
!pip install geetools
import geetools
import numpy as np
import shutil
import os
from google.colab import auth
!pip install fiona
!pip install rasterio
from rasterio.merge import merge
import rasterio as rio, fiona
import rasterio.mask as mask

     |████████████████████████████████| 16.7 MB 5.0 MB/s 
Mounted at /content/drive


Authorize the access to your Google Drive

In [ ]:
auth.authenticate_user()
from google.colab import drive
drive.mount('/content/drive')

Copy-paste your GEE link to import your account/save on the drive

In [2]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=wVyFOr4yCNOt0hrCCPi0cLoLDASpQGVAdeTNVpXoRrE&tc=58Ug1FMCCDM56v5--Z3aj4Vc90wcLI_83h1tPs0tmvg&cc=J6ZxT7G_FG9lGZQ8ZDEOdzehFl5Z9UH_L1PJ_jDqOoE

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWj3L_ANGeO9zpcdS68nU6o4c3Yw8mQQof8JU93TAmp6PH0vt1cnM9c

Successfully saved authorization token.


Functions to gather Satellite collections, filter them and download them

These functions are used to download Landsat 4-8 and Sentinel 2 images depending on a user-defined:

- time-window (from which year to which year)
- date pattern (eg: between 01/09 - 31/10 of each year)
- Area of Interest - AOI (area for which the maximum cloud percentage allows or blocks the image to be downloaded, should be smaller or equal to the ROI)
- Region of Interest - ROI (region for which the images are downloaded)
- Cloud percentage (will search for images having less or equal to this percentage of cloud covered pixels)
- Maximum cloud percentage (the cloud percentage variable will be increased percent by percent until an image is available, or until the maximum cloud percentage is reached)




In [3]:
# Depending on the year, will automatically select the available dataset between the set dates

def Satellite_collectioner(y, AOI, ROI, foldername, cloudperc, cloudmax, sdate, edate):

  # Preload the path to the dataset
  ID4 = 'LANDSAT/LT04/C01/T1_SR'
  ID5 = "LANDSAT/LT05/C01/T1_SR"
  ID7 = "LANDSAT/LE07/C01/T1_SR"
  ID8 = 'LANDSAT/LC08/C01/T1_SR'
  IDS = 'COPERNICUS/S2'


  # These loops determine which satellite collection to take
  if y < 1984:
    IDA = ID4
    Sat_ID1 = 'L4'

  elif y < 1994:
    IDA = ID4
    IDB = ID5
    Sat_ID1 = 'L4'
    Sat_ID2 = 'L5'

  elif y < 1999:
    IDA = ID5
    Sat_ID1 = 'L5'

  elif y < 2012:
    IDA = ID5
    IDB = ID7
    Sat_ID1 = 'L5'
    Sat_ID2 = 'L7'

  elif y < 2015:
    IDA = ID7
    IDB = ID8
    Sat_ID1 = 'L7'
    Sat_ID2 = 'L8'
    
  # Between 2015 and 2022, we can download Sentinel-2 images
  elif y < 2022:
    IDA = ID7
    IDB = ID8
    IDC = IDS
    Sat_ID1 = 'L7'
    Sat_ID2 = 'L8'
    Sat_ID3 = 'S2'
    
    # Initialize the percentage of clouds
    percloudS2 = 10
    cloud = 1

    # If percentage of clouds doesn't give any image, we increase it until it reaches cloudmax or finds images
    while cloud == 1 and percloudS2 < cloudmax:

      try: 
        S2st = ee.ImageCollection('COPERNICUS/S2').filterBounds(AOI).filter(ee.Filter.date(sdate, edate)).filter(ee.Filter.lessThanOrEquals('CLOUDY_PIXEL_PERCENTAGE', percloudS2))
        saverator('S2', S2st, foldername, ROI)
        cloud = 0
        print('S2 image for {0} with {1}% cloud'.format(y, percloudS2))
      except:
        percloudS2 += 1
      
    if cloud == 1:
      print('No S2 image for {0}'.format(y))
    
  # Download the Landsat collections
  try:
    collection1 = Landsater(IDA, sdate, edate)
    saverator(Sat_ID1, collection1, foldername, ROI)
    print('{0} image for {1}'.format(Sat_ID1, y))

  except: 
    print('No data for {0} {1}'.format(y, Sat_ID1))


  try:
    collection2 = Landsater(IDA, sdate, edate)
    saverator(Sat_ID2, collection2, foldername, ROI)
    print('{0} image for {1}'.format(Sat_ID2, y))

  except: 
    print('No data for {0} {1}'.format(y, Sat_ID2))


# Function to save the images on the Google Drive
def saverator(Sat_ID, collection, foldername, glob_ROI):

    geetools.batch.Export.imagecollection.toDrive(
        collection, 
        folder=foldername,  
        namePattern= Sat_ID + '_{system_date}', 
        dataType="float", 
        region=glob_ROI.bounds().getInfo()['coordinates'], 
        datePattern='y_MM_dd',
        extra=None, 
        verbose=False,
        #scale = 30   # Force the image to have a footprint of 30m
        )


# Function used to create the Landsat collection
def Landsater(satID, sdate, edate):

  collection = ee.ImageCollection(satID).filterDate(sdate, edate).filterBounds(AOI).map(maskQuality)

  return collection


# Used to compute the bits to extract for the cloud mask
def getQABits(image, start, end, mascara):
    # Compute the bits we need to extract.
    pattern = 0
    for i in range(start,end+1):
        pattern += 2**i
    # Return a single band image of the extracted QA bits, giving the band a new name.
    return image.select([0], [mascara]).bitwiseAnd(pattern).rightShift(start)


# Cloud mask
def maskQuality(image):
    # Select the QA band.
    QA = image.select('pixel_qa')
    # Get the internal_cloud_algorithm_flag bit.
    sombra = getQABits(QA,3,3,'cloud_shadow')
    nubes = getQABits(QA,5,5,'cloud')
    #  var cloud_confidence = getQABits(QA,6,7,  'cloud_confidence')
    cirrus_detected = getQABits(QA,9,9,'cirrus_detected')
    #var cirrus_detected2 = getQABits(QA,8,8,  'cirrus_detected2')
    #Return an image masking out cloudy areas.
    return image.updateMask(sombra.eq(0)).updateMask(nubes.eq(0).updateMask(cirrus_detected.eq(0)))


# Cloud mask for Sentinel-2
def maskS2clouds(image):
  QA = image.select('QA60');

  #Bits 10 and 11 are clouds and cirrus, respectively.
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11

  #Both flags should be set to zero, indicating clear conditions.
  cloud = getQABits(QA, 10, 10, 'cloud')
  cirrus = getQABits(QA, 11, 11, 'cirrus_detected')

  return image.updateMask(cloud.eq(0)).updateMask(cirrus.eq(0)).divide(10000)


# Function to move downloaded images to another folder and reorganize the data
def Mover(SAR, foldername):


  # We define the source folder to be the one in which the images were downloaded
  src_folder = '/content/drive/MyDrive/' + foldername

  # Create Optical and SAR subfolders and subsubfolders to hold the downloaded images
  if SAR:
    os.mkdir(src_folder + '/SAR')
    os.mkdir(src_folder + '/SAR/Original')
  else:
    os.mkdir(src_folder + '/Optical')
    os.mkdir(src_folder + '/Optical/Original')


  # Depending on if you are working with SAR or Optical, the destination folder will change
  if SAR:
    dst_folder = src_folder + 'SAR/Original/'
  else: 
    dst_folder = src_folder + 'Optical/Original/'

  # Search files with .txt extension in source directory
  pattern = "/*.tif"
  files = glob.glob(src_folder + pattern)

  # move the files with txt extension
  for file in files:
      # extract file name form file path
      file_name = os.path.basename(file)
      shutil.move(file, dst_folder + file_name)
      print('Moved:', file)

  return dst_folder

## Use this window to enter your parameters and download the available images over a time-period


CONFIGURE THE DOWNLOAD HERE (beware: an AOI too large will mess-up the Landsat Cloud filtering)

AOI is supposed to be smaller than the ROI

BE CAREFULE: when you enter a foldername, you can not nest the images in a smaller folder. For example, if you want to put your images in the "Images" folder located in the "Remote_Sensing" folder, you would write: foldername = "Remote_Sensing/Images/". But what the "to_drive" GEE function will do is create a folder names "Remote_Sensing/Images/". 
So what I advise is creating a separete folder that will host your downloads, clip together your images, and save the clips in the folder that you want.

In [4]:
# Define ROI
ROI = ee.Geometry.Polygon([[-141.28,59.66],
                               [-139.4,59.66],
                               [-139.4,60.7],
                               [-141.28,60.7],
                               [-141.28,59.66]]);

# Define AOI
AOI = ee.Geometry.Polygon([[-140.58164857507944,60.064393782012615],
                               [-140.1339557039857,60.064393782012615],
                               [-140.1339557039857,60.32038533124061],
                               [-140.58164857507944,60.32038533124061],
                               [-140.58164857507944,60.064393782012615]])

# Name of the folder that will host the images on your Drive
foldername = 'Projet_Remote_Sensing'
# Define a starting cloud percentage
cloudperc = 10
# Define a max cloud percentage
cloudmax = 50


# Write starting month/day and ending month/day
smd = '09-01'
emd = '10-31'

# Select starting year and ending year

syear = 2018
eyear = 2022

# Download satellite images between custom range of years
for y in range(syear, eyear+1):

  # Create the date strings
  sdate = str(y) + '-' + smd
  edate =  str(y) + '-' + emd

  # Launch the main function
  Satellite_collectioner(y, AOI, ROI, foldername, cloudperc, cloudmax, sdate, edate)

S2 image for 2018 with 10% cloud
S2 image for 2019 with 10% cloud
S2 image for 2020 with 10% cloud
S2 image for 2021 with 10% cloud


### DOWNLOADING MULTIPLE IMAGES MIGHT TAKE A FEW DAYS - COME BACK TO THIS PART AFTER ALL YOUR IMAGES HAVE BEEN DOWNLOADED

Move the images from their folder to a more organized folder

In [ ]:
# If you are working with SAR images, turn SAR to "True"
# You should already have the foldername defined as in the cells above
foldername = 'Projet_Remote_Sensing'
SAR = False

# Call the mover function to remove the images from the main directory and move them to a subsubfolder
# Keep the foldername in which you put the images, useful for later
dst_folder = Mover(SAR, foldername)

## Clip the images together

If your ROI is large, you might be overlapping over several Landsat or Sentinel paths. You might have different images covering a big region that you should clip together to transform them into 1 image.

The following code does that, by clipping together all the images from a same sensor (Landsat or Sentinel) taken the same day. 


In [5]:
# Assembler takes raster within the same defined time-window  and nanmeans them
def assembler(array_host, raster2, lim_band):

  # Open the 2nd raster to merge as an array
  raster2 = rio.open(raster2, 'r').read()

  # Create a host array that will be the nanmean of the two other rasters. lim_band is the amount of band per sensor 
  raster = np.zeros((lim_band, array_host.shape[1], array_host.shape[2]))

  # Fill-in the host array with the different images
  for i in range(lim_band):
      raster[i, :, :] = np.nanmean(np.dstack((array_host[i, :, :], raster2[i,:,:])),2)

  return(raster)
  

# Merges a variable amount of rasters, clips them with the outline, and saves them as tifs
def merger(list_similar, name_merge, folder):

  # Initialize the list that will host all the rasters from the same day/satellite
  rasters = []

  # Put all these images in the same list
  for i in list_similar:

    raster = rio.open(i, "r+", nodata = np.nan)
    rasters.append(rio.open(i, "r+", nodata = np.nan))

  # Kepp the metadata of a raster as a template (they all have the same CRS and extent)
  out_meta = raster.meta

  # Use the "merge" function from rasterio to merge all rasters
  mosaic, output = merge(rasters)

  # Write the "mosaic" array as a raster so it can be called for the function clipping the glacier mask to it
  with rio.open(folder + '/' + name_merge + '.tif', "w", **out_meta) as dest:
      dest.write(mosaic)

  # Open this array as a raster, overwrite the previous array to save memory
  mosaic = rio.open(folder + '/' + name_merge + '.tif', "r+", nodata = np.nan)

  # Use the "Clipper" function to clip to the merged raster the glacier mask (everything outside of glacier mask is NaN)
  mosaic, passaran = Clipper(mosaic, 'Glacier')

  # If the file is empty (passaran = 0): delete the tif in the output folder
  if passaran == 1:
    with rio.open(folder + '/' + name_merge + '.tif', "w", **out_meta) as dest:
          dest.write(mosaic)
  
  else:
    os.remove(folder + '/' + name_merge + '.tif')

  
# Clip Randolf's outline
def Clipper(raster, option, metadata):

  # We define different "zones" to clip because we are working with 3 main tributaries.
  # If we don't do that, the ELA searching algorithm can give messed up results. 

  # Used to clip the raster to the glacier's extend 
  if option == 'Glacier':
    path = '/content/drive/MyDrive/Projet_Remote_Sensing/Outlines/glacier.shp'

  # Used to calculate an average value in the accumulation area of the Agassiz    
  elif option == 'Agassiz': 
    path = '/content/drive/MyDrive/Projet_Remote_Sensing/Outlines/Shapes/Acc/Acc_Agassiz.shp'

  # Used to calculate an average value in the accumulation area of the Seward  
  elif option == 'Seward':
    path = '/content/drive/MyDrive/Projet_Remote_Sensing/Outlines/Shapes/Acc/Acc_Seward.shp'

  # Used to calculate an average value in the accumulation area of the Marvine
  elif option == 'Marvine':
    path = '/content/drive/MyDrive/Projet_Remote_Sensing/Outlines/Shapes/Acc/Acc_Marvine.shp'

  # Is used to calculate an average value in the ablation shape
  elif option == 'Ablation':
    path = '/content/drive/MyDrive/Projet_Remote_Sensing/Outlines/Shapes/Abl/Abl.shp'

  
  # Read Shapefile
  with fiona.open(path, "r") as shapefile:
      shapes = [feature["geometry"] for feature in shapefile]


  # read imagery file
  out_image, out_transform = mask.mask(raster, shapes, crop=True, nodata=np.nan)

  # Check that after the clip, the image is not empty
  test = out_image[~np.isnan(out_image)]

  if test[test > 0].shape[0] == 0:
      raise RuntimeError("Empty output")

  out_meta = raster.profile
  out_meta.update({"height": out_image.shape[1],
                    "width": out_image.shape[2],
                    "transform": out_transform})


  if test[test>0].shape[0] == 0:
    passaran = 0
  else:
    passaran = 1

  # convert the zeros into NaNs
  
  return(out_image, passaran, metadata)


# This function will allow to merge/clip optical images (only merge or clip) or simply clip SAR images
# (SAR images are already downloaded from the same path, so no merging is necessary)
def Master_Clipper(list_names, path, SAR):

  if SAR:
      # Read glacier outline file
      with fiona.open('/content/drive/MyDrive/Projet_Remote_Sensing/Outlines/glacier.shp', "r") as shapefile:
        shapes = [feature["geometry"] for feature in shapefile]

  # Initialize the counter of the loop      
  i = 0

  while i < len(list_names):
    # If SAR
    if SAR:
      # Get the names in the SAR images folder
      name = list_names[i].split('/')[-1][:8]

      # Open the first image as a reference to grab its metadata
      raster = rio.open(list_names[i], "r+", nodata = np.nan)
      out_meta = raster.meta

      # read imagery file and clip the glacier extent
      out_image, out_transform = rio.mask.mask(raster, shapes, crop=True, nodata = np.nan)

      # Check that after the clip, the image is not empty
      test = out_image[~np.isnan(out_image)]

      # If the image is empty, it will be deleted
      if test[test>0].shape[0] == 0:
        passaran = 0
      else:
        passaran = 1

      if passaran == 1:

        with rio.open(path + '/' + name + '.tif', "w", **out_meta) as dest:
              dest.write(out_image)

      else:
        os.remove(path + '/' + name + '.tif')

      # We increment the loop
      i += 1

    # If Optical
    else:

      # First, gather all the images from the same months
      list_similar = sorted(glob.glob(list_names[i].split('_')[0] + '_' + list_names[i].split('_')[1] + '_' + list_names[i].split('_')[2] + '_' + list_names[i].split('_')[3] + '_' + list_names[i].split('_')[4] +'*'))
      name = list_similar[-1].split('/')[-1]

      # Merge Rasters with same date  
      try: 
        merger(list_similar, name, path)
        print('File {0} merged successfully'.format(i))
      except:
        print('File {0} could not be processed'.format(i))
      
      # Increment the loop by the length of files with the same name for a given day
      i += len(list_similar)
  

The following function call will first merge all rasters becoming to the same satellite taken on the same day, then it will clip them all to the Glacier Mask from the Randolph Glacier inventory.

Be aware, I encountered issues while clipping the glacier mask. Sometimes it works and sometimes it does not, for reasons unknown and by using the exact same lines of code both times: [GIS stack topic](https://gis.stackexchange.com/questions/429578/python-cropping-a-tiff-with-shapefile-shifts-the-output-slightly-to-the-east/429586#429586)

In [ ]:
# Loop that gathers rasters with the same date from the destination folder defined during the moving of the images
list_names = sorted(glob.glob(dst_folder + '*'))

# Depending on if you use SAR images, the path will be automatically determined
if SAR:
  path = '/content/drive/MyDrive/Projet_Remote_Sensing/SAR/Clipped_Images'
else: 
  path = '/content/drive/MyDrive/Projet_Remote_Sensing/Optical/Clipped_Images'

Master_Clipper(list_names, path, SAR)